# Predict

**调用训练好的模型，预测图片中害虫位置及类别**

**测试数据1**

## 一、测试数据准备

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from cv2 import cv2
import pandas as pd

In [2]:
csv_path = 'datasets/test.csv'
imgs_dir = 'datasets/tdcup/images'

df = pd.read_csv(csv_path, encoding='gbk')  # (802, 2)

imgs_name = df.iloc[:, 1].values  # (802,)

## 二、加载模型

In [4]:
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from mpvit import add_mpvit_config

exp_tag = 'base_thred60_data_fixed'

cfg = get_cfg()
add_mpvit_config(cfg)
cfg.merge_from_file('configs/maskrcnn/mask_rcnn_mpvit_base_ms_3x.yaml')
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.6
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 28
cfg.MODEL.WEIGHTS = 'output/mask_rcnn_mpvit_base_ms_3x/exp2/model_0019999.pth'
cfg.freeze()


In [5]:
from detectron2.data import MetadataCatalog, DatasetCatalog


# 获取数据集元数据
pest_metadata = MetadataCatalog.get('pest_train')  #  property: ['json_file', 'name', 'set', 'thing_classes', 'thing_dataset_id_to_contiguous_id']
pest_train_dataset = DatasetCatalog.get('pest_train')  # list of dict
classes_name = pest_metadata.thing_classes

print(classes_name)


Category ids in annotations are not in [1, #categories]! We'll apply a mapping for you.



['6', '7', '8', '9', '10', '25', '41', '105', '110', '115', '148', '156', '222', '228', '235', '256', '280', '310', '387', '392', '394', '398', '401', '402', '430', '480', '485', '673']


In [6]:
# 创建必要的文件夹

output_dir = os.path.join('results/test1', exp_tag)
imgs_save_dir = os.path.join(output_dir, 'pred_imgs')

os.makedirs(output_dir, exist_ok=True)  # output path
os.makedirs(imgs_save_dir, exist_ok=True)  # output path

## 二、预测

In [7]:
# cnt = 1

# for name_ in imgs_name:
#     path_ = os.path.join(imgs_dir, name_)

#     img = plt.imread(path_)
#     # plt.imshow(img)
#     if not img.shape:
#         continue

#     img_ = cv2.resize(img, pipeline_is)
#     flag = pipeline.predict(img_.flatten().reshape(1, -1))[0]
#     if flag:
#         cnt += 1


# print(f'{cnt} images contain objects, all {len(imgs_name)} images')
# 180 images contain objects, all 802 images

In [8]:
predictor = DefaultPredictor(cfg)

In [10]:
from detectron2.utils.visualizer import Visualizer

import shutil
import torch

result_list = []
cnt = 1


for name_ in imgs_name:
    path_ = os.path.join(imgs_dir, name_)
    img_save_path = os.path.join(imgs_save_dir, name_)

    img = plt.imread(path_)
    
    if sum(img.shape) <= 3:
        result_list.append([cnt, name_, 0, 0, 0, 0, 0, 0, 0])
        cnt += 1
        continue    

    # todo:
    # using classifier determine whether an image includes object or not
    # img_ = cv2.resize(img, pipeline_is)
    # flag = pipeline.predict(img_.flatten().reshape(1, -1))[0]
    # if not flag:
    #     # shutil.copy(path_, img_save_path)
    #     continue
    

    output = predictor(img)
    instances = output["instances"]

    pred_classes = instances.pred_classes
    pred_boxes = instances.pred_boxes

    # continue when detect nothing
    if pred_classes.shape[0] == 0:
        result_list.append([cnt, name_, 0, 0, 0, 0, 0, 0, 0])
        cnt += 1
        continue

    # save predicted images
    v = Visualizer(img, MetadataCatalog.get(cfg.DATASETS.TRAIN[0]))
    out = v.draw_instance_predictions(instances.to("cpu"))
    pred_img = out.get_image()
    
    plt.imsave(img_save_path, pred_img)

    # save predicted results to csv file
    pred_classes = torch.tensor([int(classes_name[i]) for i in pred_classes], device=pred_classes.device)
    for class_, box_ in zip(pred_classes, pred_boxes):
        tpx, tpy, brx, bry = box_
        mx, my = (tpx + brx) / 2, (tpy + bry) / 2
        temp = [cnt, name_, class_.item(), mx.item(), my.item(), tpx.item(), tpy.item(), brx.item(), bry.item()]
        result_list.append(temp)
        cnt += 1
    

/root/anaconda3/envs/detectron2/lib/python3.8/site-packages/detectron2/structures/image_list.py:88: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  max_size = (max_size + (stride - 1)) // stride * stride
/root/anaconda3/envs/detectron2/lib/python3.8/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


## 三、保存结果

In [11]:
# save result2.csv
save_path = os.path.join(output_dir, 'result2.csv')

df = pd.DataFrame(np.array(result_list))

header = ['序号','文件名','虫子编号','中心点x坐标','中心点y坐标','左上角x坐标','左上角y坐标','右下角x坐标','右下角y坐标']

df.to_csv(save_path, header=header,
                        index=None, encoding='gbk')

In [ ]:
# save result3.csv

save3_path = os.path.join(output_dir, 'result3.csv')

res_dict = {}  # key为图像名称，value为字典（key为类别编号，value为该类别目标数量）
for item_ in result_list:
    name_, cid = item_[1:3]

    if name_ not in res_dict.keys():
        res_dict[name_] = {}
    
    if cid not in res_dict[name_].keys():
        res_dict[name_][cid] = 0

    if cid != 0:
        res_dict[name_][cid] += 1


res3 = []
for name_, v in res_dict.items():
    for kk, vv in v.items():
        res3.append([name_, kk, vv])

idx = (np.arange(len(res3)) + 1).reshape(-1, 1)

data_new = np.hstack((idx, np.array(res3)))

df_new = pd.DataFrame(data_new)

header = ['序号','文件名','虫子编号','数量']

df_new.to_csv(save3_path, header=header,
                        index=False, encoding='gbk')